# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(5):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [55]:
def get_graph(nodes_number):
    edges = pd.read_csv('datasets/wisconsin.csv', header=None).iloc[1:,]
    edges = edges.astype('int')
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[0]):
        graph_np[edges.iloc[i, 0], edges.iloc[i, 1]]=1
        graph_np[edges.iloc[i, 1], edges.iloc[i, 0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*6,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [56]:
nodes_number = 251

In [58]:
embedding_dim = 12
n_clusters = 88   # 最好是48
alpha = 1
beta = 4.6
DEC = 4.2   # 现在最好的是4.2
LR = 0.1

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask = get_graph(nodes_number)
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.5912759953022194
AUC SCORE:  0.5629629629629629
AP：  0.5505894727465028
AUC SCORE:  0.5441975308641975
######################### Module cycle ： 1 ##########################
AP：  0.5609066677457744
AUC SCORE:  0.5743209876543209
AP：  0.5711215685600498
AUC SCORE:  0.5876543209876542
######################### Module cycle ： 2 ##########################
AP：  0.5847780545888346
AUC SCORE:  0.5980246913580246
AP：  0.5972267630501472
AUC SCORE:  0.597037037037037
######################### Module cycle ： 3 ##########################
AP：  0.5877182773638198
AUC SCORE:  0.5911111111111111
AP：  0.5948545474525114
AUC SCORE:  0.5930864197530864
######################### Module cycle ： 4 ##########################
AP：  0.5981300753170423
AUC SCORE:  0.5891358024691359
AP：  0.6121724703059751
AUC SCORE:  0.5920987654320987
######################### Module cycle ： 5 ##########################
AP：  0.611773697253064
AUC SCOR

AP：  0.74386940480872
AUC SCORE:  0.6938271604938272
AP：  0.7468193869431132
AUC SCORE:  0.6958024691358023
######################### Module cycle ： 47 ##########################
AP：  0.7486644518441787
AUC SCORE:  0.6987654320987654
AP：  0.7493719416288521
AUC SCORE:  0.6997530864197531
######################### Module cycle ： 48 ##########################
AP：  0.7500178034935745
AUC SCORE:  0.7012345679012345
AP：  0.7509752122501646
AUC SCORE:  0.7017283950617285
######################### Module cycle ： 49 ##########################
AP：  0.7516320048717525
AUC SCORE:  0.702716049382716
AP：  0.7520925708160187
AUC SCORE:  0.70320987654321
######################### Module cycle ： 50 ##########################
AP：  0.7527158194202314
AUC SCORE:  0.7041975308641976
AP：  0.7530668429285863
AUC SCORE:  0.7041975308641976
######################### Module cycle ： 51 ##########################
AP：  0.7550307846983968
AUC SCORE:  0.7061728395061729
AP：  0.7552794597059946
AUC SCORE:  0.7066666

AP：  0.8332812238329674
AUC SCORE:  0.8044444444444445
######################### Module cycle ： 93 ##########################
AP：  0.833781724333468
AUC SCORE:  0.8049382716049382
AP：  0.8330640021319832
AUC SCORE:  0.8049382716049384
######################### Module cycle ： 94 ##########################
AP：  0.8337047252537553
AUC SCORE:  0.8059259259259259
AP：  0.8341442009015643
AUC SCORE:  0.8069135802469136
######################### Module cycle ： 95 ##########################
AP：  0.8348406238202094
AUC SCORE:  0.8074074074074074
AP：  0.8345712635508491
AUC SCORE:  0.8069135802469136
######################### Module cycle ： 96 ##########################
AP：  0.8339623735301944
AUC SCORE:  0.8069135802469135
AP：  0.8336573626369481
AUC SCORE:  0.8064197530864198
######################### Module cycle ： 97 ##########################
AP：  0.8344512665887469
AUC SCORE:  0.8079012345679013
AP：  0.8346688268063072
AUC SCORE:  0.808395061728395
######################### Module cycle ： 9

AP：  0.8450131418626261
AUC SCORE:  0.8271604938271605
######################### Module cycle ： 139 ##########################
AP：  0.8458198476041021
AUC SCORE:  0.8281481481481482
AP：  0.8458198476041021
AUC SCORE:  0.8281481481481482
######################### Module cycle ： 140 ##########################
AP：  0.8460940178782724
AUC SCORE:  0.8286419753086419
AP：  0.8463990287715185
AUC SCORE:  0.8291358024691358
######################### Module cycle ： 141 ##########################
AP：  0.8460940178782724
AUC SCORE:  0.8286419753086419
AP：  0.8454019945856299
AUC SCORE:  0.8281481481481482
######################### Module cycle ： 142 ##########################
AP：  0.8450969836923837
AUC SCORE:  0.8276543209876542
AP：  0.8456735066992472
AUC SCORE:  0.8286419753086419
######################### Module cycle ： 143 ##########################
AP：  0.8452137841323021
AUC SCORE:  0.8276543209876543
AP：  0.8453679033931752
AUC SCORE:  0.8276543209876543
######################### Module cy

AUC SCORE:  0.8296296296296296
######################### Module cycle ： 184 ##########################
AP：  0.8463507837693559
AUC SCORE:  0.8301234567901234
AP：  0.8460479652275437
AUC SCORE:  0.8296296296296296
######################### Module cycle ： 185 ##########################
AP：  0.8463507837693559
AUC SCORE:  0.8301234567901234
AP：  0.8462259398242873
AUC SCORE:  0.8296296296296296
######################### Module cycle ： 186 ##########################
AP：  0.8462259398242873
AUC SCORE:  0.8296296296296296
AP：  0.8462259398242873
AUC SCORE:  0.8296296296296296
######################### Module cycle ： 187 ##########################
AP：  0.8465566276549752
AUC SCORE:  0.8301234567901234
AP：  0.8463322686806161
AUC SCORE:  0.8296296296296296
######################### Module cycle ： 188 ##########################
AP：  0.8463322686806161
AUC SCORE:  0.8296296296296296
AP：  0.8463322686806161
AUC SCORE:  0.8296296296296296
######################### Module cycle ： 189 ##############

...

AP：  0.8329376824713487
AUC SCORE:  0.7935802469135802
######################### Module cycle ： 84 ##########################
AP：  0.8331931103129835
AUC SCORE:  0.794074074074074
AP：  0.8340180551138319
AUC SCORE:  0.794074074074074
######################### Module cycle ： 85 ##########################
AP：  0.8342641915253874
AUC SCORE:  0.7945679012345679
AP：  0.8369479501527916
AUC SCORE:  0.7960493827160493
######################### Module cycle ： 86 ##########################
AP：  0.8385872255250375
AUC SCORE:  0.7975308641975308
AP：  0.8384501693638848
AUC SCORE:  0.797037037037037
######################### Module cycle ： 87 ##########################
AP：  0.8400585473378226
AUC SCORE:  0.7990123456790124
AP：  0.839933703392754
AUC SCORE:  0.7985185185185185
######################### Module cycle ： 88 ##########################
AP：  0.8416430691298975
AUC SCORE:  0.7999999999999999
AP：  0.8431348612883562
AUC SCORE:  0.8009876543209876
######################### Module cycle ： 89 

AP：  0.8581774142417358
AUC SCORE:  0.8241975308641976
######################### Module cycle ： 130 ##########################
AP：  0.8581774142417358
AUC SCORE:  0.8241975308641976
AP：  0.8577358188001404
AUC SCORE:  0.8237037037037037
######################### Module cycle ： 131 ##########################
AP：  0.8582099771690357
AUC SCORE:  0.8241975308641976
AP：  0.8587214410138329
AUC SCORE:  0.8246913580246913
######################### Module cycle ： 132 ##########################
AP：  0.8590757083246219
AUC SCORE:  0.8251851851851851
AP：  0.8583061075550211
AUC SCORE:  0.8241975308641976
######################### Module cycle ： 133 ##########################
AP：  0.8591824595941638
AUC SCORE:  0.8251851851851852
AP：  0.8590968838174955
AUC SCORE:  0.8251851851851852
######################### Module cycle ： 134 ##########################
AP：  0.8590968838174955
AUC SCORE:  0.8251851851851852
AP：  0.8595384792590909
AUC SCORE:  0.8256790123456791
######################### Module cy

AP：  0.8703371387318691
AUC SCORE:  0.8350617283950618
AP：  0.8707487945495966
AUC SCORE:  0.8355555555555556
######################### Module cycle ： 177 ##########################
AP：  0.8707487945495966
AUC SCORE:  0.8355555555555556
AP：  0.8707487945495966
AUC SCORE:  0.8355555555555556
######################### Module cycle ： 178 ##########################
AP：  0.8706308375711744
AUC SCORE:  0.8355555555555556
AP：  0.8699078568481937
AUC SCORE:  0.8345679012345679
######################### Module cycle ： 179 ##########################
AP：  0.870189242129579
AUC SCORE:  0.8350617283950617
AP：  0.8699078568481937
AUC SCORE:  0.8345679012345679
######################### Module cycle ： 180 ##########################
AP：  0.8700315469617728
AUC SCORE:  0.8345679012345679
AP：  0.8700315469617728
AUC SCORE:  0.8345679012345679
######################### Module cycle ： 181 ##########################
AP：  0.8708868224052835
AUC SCORE:  0.8355555555555556
AP：  0.8713204267413268
AUC SCORE:  

AP：  0.873119036589049
AUC SCORE:  0.8375308641975309
AP：  0.873119036589049
AUC SCORE:  0.8375308641975309
######################### Module cycle ： 224 ##########################
AP：  0.8727871257308224
AUC SCORE:  0.8365432098765432
AP：  0.8727871257308224
AUC SCORE:  0.8365432098765432
######################### Module cycle ： 225 ##########################
AP：  0.873136596385566
AUC SCORE:  0.837037037037037
AP：  0.873136596385566
AUC SCORE:  0.837037037037037
######################### Module cycle ： 226 ##########################
AP：  0.8733855338476182
AUC SCORE:  0.837530864197531
AP：  0.873136596385566
AUC SCORE:  0.837037037037037
######################### Module cycle ： 227 ##########################
AP：  0.8733855338476182
AUC SCORE:  0.837530864197531
AP：  0.8733855338476182
AUC SCORE:  0.837530864197531
######################### Module cycle ： 228 ##########################
AP：  0.8740952961847317
AUC SCORE:  0.8385185185185184
AP：  0.8743871031432054
AUC SCORE:  0.83901234

AP：  0.8749675541301505
AUC SCORE:  0.8390123456790124
AP：  0.8749675541301505
AUC SCORE:  0.8390123456790124
######################### Module cycle ： 272 ##########################
AP：  0.8749675541301505
AUC SCORE:  0.8390123456790124
AP：  0.8749675541301505
AUC SCORE:  0.8390123456790124
######################### Module cycle ： 273 ##########################
AP：  0.8749675541301505
AUC SCORE:  0.8390123456790124
AP：  0.8746491361646738
AUC SCORE:  0.8385185185185187
######################### Module cycle ： 274 ##########################
AP：  0.8746491361646738
AUC SCORE:  0.8385185185185187
AP：  0.8746491361646738
AUC SCORE:  0.8385185185185187
######################### Module cycle ： 275 ##########################
AP：  0.8746491361646738
AUC SCORE:  0.8385185185185187
AP：  0.8750307298796007
AUC SCORE:  0.8390123456790124
######################### Module cycle ： 276 ##########################
AP：  0.8750307298796007
AUC SCORE:  0.8390123456790124
AP：  0.8750307298796007
AUC SCORE: 

In [59]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.8311402343793037
AP FIRST AP STD:  0.037545899223665816
AP FIRST AUC MEAN:  0.8097777777777777
AP FIRST AUC STD:  0.033264895450289676
AUC FIRST AP MEAN:  0.8297647888441702
AUC FIRST AP STD:  0.03817070092363526
AUC FIRST AUC MEAN:  0.8108641975308641
AUC FIRST AUC STD:  0.03330094448391526
